# **Font Effects using StableDiffusion**

## Imports

In [ ]:
!pip install diffusers
!pip install transformers scipy ftfy accelerate
!pip install "ipywidgets>=7,<8"

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline, DPMSolverMultistepScheduler
from PIL import Image, ImageDraw, ImageFont
import numpy as np

## Use GPU if available

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    print("GPU NOT AVAILABLE!!!")

print(f"device = {device}")

## Create image of written words

In [ ]:
image_config = {
    "size": (512, 512),
    "background_color": (255, 255, 255),
    "text_color": (0, 0, 0),
    "text_thickness": 7
}

In [ ]:
def create_text_img(text, font_size=150, font_type="./Roboto-Bold.ttf"):
    """Returns PIL img with text on white background"""

    # make text uppercase => better results
    # text = text.upper()

    # choose font
    font = ImageFont.truetype(font_type, size=font_size)
    # font = ImageFont.load_default()

    # geneeate white blank image
    image = Image.new("RGB", size=image_config["size"], color=image_config["background_color"])

    # calculate text position -> center of image
    text_size = font.getbbox(text)
    text_x = (image.width - text_size[2]) / 2
    text_y = (image.height - text_size[3]) / 2

    # thicken text by overlapping multiple layers of text
    thickness = image_config["text_thickness"]
    thickness_step = 3

    draw = ImageDraw.Draw(image)
    for dx in range(-thickness, thickness + 1, thickness_step):
        for dy in range(-thickness, thickness + 1, thickness_step):
            draw.text((text_x + dx, text_y + dy), text=text, fill=image_config["text_color"], font=font, spacing=50)

    return image

def create_text_mask(image):
    """Returns PIL image of mask on letters"""

    # generate numpy array from grayscale img
    grayscale_image = image.convert("L")
    grayscale_array = np.array(grayscale_image)

    threshold = 128

    # generate mask
    mask_array = np.where(grayscale_array > threshold, 0, 255).astype(np.uint8)

    return Image.fromarray(mask_array)

def no_safety_checker(images, clip_input):
    return images, [False] * len(images)


Create image and visualize it!

In [ ]:
text = "E"
img_name = text + "_img"
img_folder = ""

# create text image
# img = create_text_img(text=text, font_size=450, font_type="/content/DMSerifDisplay-Regular.ttf")
img = create_text_img(text=text, font_size=450, font_type="/content/LilitaOne-Regular.ttf")

# create mask of image
img_mask = create_text_mask(img)

# save image and mask of it
img.save(img_folder + img_name + ".png")
img_mask.save(img_folder + img_name + "_mask" + ".png")

img

## Initialize inpainting pipeline

In [ ]:
inpaint_model_path = "runwayml/stable-diffusion-inpainting"
inpaint_model_path2 = "stabilityai/stable-diffusion-2-inpainting"

pipeInpaint = StableDiffusionInpaintPipeline.from_pretrained(inpaint_model_path2, torch_dtype=torch.float32)
pipeInpaint.scheduler = DPMSolverMultistepScheduler.from_config(pipeInpaint.scheduler.config)
pipeInpaint = pipeInpaint.to(device)

In [ ]:
# Kadinksy Model => doesn t work!!
# from diffusers import AutoPipelineForInpainting
# kadinsky_model_path = "kandinsky-community/kandinsky-2-2-decoder-inpaint"
# pipeInpaint = AutoPipelineForInpainting.from_pretrained(kadinsky_model_path, torch_dtype=torch.float16)


## Generate Font effect

In [ ]:
def create_font_effect(prompt, image, mask, file_name=""):
    pipeInpaint.safety_checker = no_safety_checker
    output_img = pipeInpaint(
                    prompt=prompt,
                    image=image,
                    mask_image=mask,
                    num_inference_steps=50).images[0]
    if file_name == "":
      file_name = "_".join(prompt.split(" "))
    output_img.save(file_name + ".png")
    print(f"image saved to {file_name}")

    return output_img


## Final result

In [ ]:
import random
import os

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


In [ ]:
# set_seed(43)
prompt = "red golden gate bridge"
result = create_font_effect(prompt=prompt, image=img, mask=img_mask)

result